In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
df = pd.read_csv('./FacialExpressions_dataset.csv')

In [3]:
df.head()

,Image ID,Attentative Score/0-10,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,1User1_10,10.0,NaN,NaN,NaN
1,2User1_5,5.0,NaN,NaN,NaN
2,3User1_9,9.0,NaN,NaN,NaN
3,4User1_8,8.0,NaN,NaN,NaN
4,5User1_4,4.0,NaN,NaN,NaN


In [4]:
parent = '/home/rt/Documents/gayan/attentiveScore/facialExpressions/FacialExpressionDataset/FacialExpressions_800_800/'


In [10]:
rows = []
df2 = []
lines = []
for i, row in df.iterrows():
    if(i<2):
        image = row['Image ID'] + '.jpg'
        k = 'cat "' + parent + image + '"| docker run -i think/affectiva'
        print(k)
        output = execCommand('cat "' + parent + image + '"| docker run -i think/affectiva')
#         output = lines
        lines = output.split('\n')
        headers = lines[0].split(',')
        if(i==0):
            df2 = pd.DataFrame(columns=[headers])
            df2['imageId'] = 'i'
        lines = output.split('\n')
        values = lines[1].split(',')
        values.append(row['Image ID'])
#         values = np.array(lines[1].split(','))
#         values = values.reshape(1,54)
        print(len(values))
        print(len(headers))
        if(len(values) == len(headers)+1):
            df2.loc[i] = values
        else:
            print(values)
#         df2 = pd.concat([df2, values])
#         break
# print(headers)
# print(rows)
# df2 = pd.DataFrame(rows, columns=headers)
print(df2.head())
# print(df2.columns)
# print(df2['attention'])

cat "/home/rt/Documents/gayan/attentiveScore/facialExpressions/FacialExpressionDataset/FacialExpressions_800_800/1User1_10.jpg"| docker run -i think/affectiva
55
54
cat "/home/rt/Documents/gayan/attentiveScore/facialExpressions/FacialExpressionDataset/FacialExpressions_800_800/2User1_5.jpg"| docker run -i think/affectiva
55
54
  TimeStamp faceId interocularDistance glasses    age    ethnicity gender  \
0    0.0000      0            245.7521      no  18-24  south asian   male   
1    0.0000      0            162.5996      no  35-44    caucasian   male   

  dominantEmoji     pitch     yaw  ... disappointed    rage   smirk    wink  \
0       Unknown  -23.5757  6.8450  ...       0.0018  0.0040  0.0000  0.0018   
1       Unknown   -1.6601  5.1657  ...       0.0018  0.0129  0.0000  0.0036   

  stuckOutTongueWinkingEye stuckOutTongue flushed  scream      imageId  
0                   2.2981         2.2981  0.0018  0.2483    1User1_10  
1                   2.3035         2.3035  0.0018  0.06

In [6]:
def execCommand(cmd):
    stream = os.popen(cmd)
    output = stream.read()
    return output

In [ ]:
# execCommand('pwd')

In [ ]:
# execCommand('ls')

In [9]:
print(df2.head())

Empty DataFrame
Columns: [(T,), (imageId,)]
Index: []


In [ ]:
df2.to_csv('out2.csv', index=False)


In [10]:
df2 = pd.read_csv('./out2.csv')

In [4]:
df2['attentiveness'] = 0

In [9]:
for i, r in df2.iterrows():
    for index, row in df.iterrows():
        if(row['Image ID'] == r['imageId']):
            score = 1
            if(row['Attentative Score/0-10'] != 0):
                score = row['Attentative Score/0-10']
            df2.loc[i, 'attentiveness'] = score
            break

In [ ]:
df2.head()

In [ ]:
df2.columns

In [ ]:
df2.to_csv('out2.csv', index=False)

In [2]:
# import findspark
# findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)

face_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('out2.csv')
face_df.take(1)

[Row(TimeStamp=0.0, faceId=0, interocularDistance=245.7521, glasses='no', age='18-24', ethnicity='south asian', gender='male', dominantEmoji='Unknown', pitch=-23.5757, yaw=6.845, roll=-0.1539, joy=0.0016, fear=0.0059, disgust=0.4212, sadness=0.0251, anger=0.004, surprise=0.2828, contempt=0.1928, valence=-4.4333, engagement=3.8688, smile=0.0, innerBrowRaise=3.7619, browRaise=3.8688, browFurrow=0.0004, noseWrinkle=0.0008, upperLipRaise=0.2653, lipCornerDepressor=0.0382, chinRaise=3.9751, lipPucker=3.5008, lipPress=6.1959, lipSuck=0.3115, mouthOpen=0.077, smirk=0.0038, eyeClosure=0.0, attention=95.3955, eyeWiden=0.0455, cheekRaise=0.0, lidTighten=0.0001, dimpler=41.4116, lipStretch=0.0001, jawDrop=59.8029, relaxed=0.0016, smiley=0.0016, laughing=0.0016, kissing=0.0035, disappointed=0.0018, rage=0.004, smirk.1=0.0, wink=0.0018, stuckOutTongueWinkingEye=2.2981, stuckOutTongue=2.2981, flushed=0.0018, scream=0.2483, Unnamed: 53=None, imageId='1User1_10', attentiveness=10.0)]

In [2]:
# !pip3 install pyspark
# house_df.select('gender').distinct().collect()

In [3]:
# house_df.describe().toPandas().transpose()

In [4]:
# print(house_df.columns)

In [5]:
# from pyspark.ml.feature import StringIndexer
# indexer = StringIndexer(inputCol="gender", outputCol="gender_new")
# house_df = indexer.fit(house_df).transform(house_df)
# house_df.take(100)

In [3]:
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler, MinMaxScaler


vectorAssembler = VectorAssembler(inputCols = ['joy', 'fear', 'disgust', 'sadness', 'anger', 'surprise', 'contempt', 'valence', 'engagement', 'smile', 'innerBrowRaise', 'browRaise', 'browFurrow', 'noseWrinkle', 'upperLipRaise', 'lipCornerDepressor', 'chinRaise', 'lipPucker', 'lipPress', 'lipSuck', 'mouthOpen', 'smirk', 'eyeClosure', 'eyeWiden', 'cheekRaise', 'lidTighten', 'dimpler', 'lipStretch', 'jawDrop'], outputCol = 'features')
vface_df = vectorAssembler.transform(face_df)
vface_df.take(1)

[Row(TimeStamp=0.0, faceId=0, interocularDistance=245.7521, glasses='no', age='18-24', ethnicity='south asian', gender='male', dominantEmoji='Unknown', pitch=-23.5757, yaw=6.845, roll=-0.1539, joy=0.0016, fear=0.0059, disgust=0.4212, sadness=0.0251, anger=0.004, surprise=0.2828, contempt=0.1928, valence=-4.4333, engagement=3.8688, smile=0.0, innerBrowRaise=3.7619, browRaise=3.8688, browFurrow=0.0004, noseWrinkle=0.0008, upperLipRaise=0.2653, lipCornerDepressor=0.0382, chinRaise=3.9751, lipPucker=3.5008, lipPress=6.1959, lipSuck=0.3115, mouthOpen=0.077, smirk=0.0038, eyeClosure=0.0, attention=95.3955, eyeWiden=0.0455, cheekRaise=0.0, lidTighten=0.0001, dimpler=41.4116, lipStretch=0.0001, jawDrop=59.8029, relaxed=0.0016, smiley=0.0016, laughing=0.0016, kissing=0.0035, disappointed=0.0018, rage=0.004, smirk.1=0.0, wink=0.0018, stuckOutTongueWinkingEye=2.2981, stuckOutTongue=2.2981, flushed=0.0018, scream=0.2483, Unnamed: 53=None, imageId='1User1_10', attentiveness=10.0, features=DenseVect

In [4]:
# normalizer = Normalizer(inputCol="features", outputCol="scaledFeatures", p=1.0)
# vhouse_df = normalizer.transform(vhouse_df)

# scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

# Compute summary statistics and generate MinMaxScalerModel
# scalerModel = scaler.fit(vhouse_df)
# vhouse_df = scalerModel.transform(vhouse_df)


# vhouse_df = vhouse_df.select(['scaledFeatures', 'attentiveness'])
vface_df = vface_df.select(['features', 'attentiveness'])
vface_df.show(3)

+--------------------+-------------+
|            features|attentiveness|
+--------------------+-------------+
|[0.0016,0.0059,0....|         10.0|
|[0.0013,0.006,0.4...|          5.0|
|[0.0018,0.0052,0....|          9.0|
+--------------------+-------------+
only showing top 3 rows



In [5]:
splits = vface_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [10]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='attentiveness', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
# print("Coefficients: " + str(lr_model.coefficients))
# print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.003885541497127465,0.0,0.0,-0.008099264668841916,0.0,-0.009035215939855653,0.0,0.0,-3.994049905867132e-05,0.0,0.0,0.0,0.0,0.0,-0.044613367254392515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.010299957759388445]
Intercept: 6.147538542785145


In [11]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 3.084876
r2: 0.152488


In [12]:
train_df.describe().show()

+-------+------------------+
|summary|     attentiveness|
+-------+------------------+
|  count|               739|
|   mean|5.7347767253044655|
| stddev| 3.353196153217222|
|    min|               1.0|
|    max|              10.0|
+-------+------------------+



In [9]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","attentiveness","features").show(5)

from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="attentiveness",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+-----------------+-------------+----------+
|       prediction|attentiveness|  features|
+-----------------+-------------+----------+
|6.117771779183713|          1.0|(34,[],[])|
|6.117771779183713|          1.0|(34,[],[])|
|6.117771779183713|          1.0|(34,[],[])|
|6.117771779183713|          1.0|(34,[],[])|
|6.117771779183713|          6.0|(34,[],[])|
+-----------------+-------------+----------+
only showing top 5 rows

R Squared (R2) on test data = 0.160105


In [10]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 3.05679


In [12]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","attentiveness","features").show()

+------------------+-------------+--------------------+
|        prediction|attentiveness|            features|
+------------------+-------------+--------------------+
| 6.117771779183713|          1.0|          (34,[],[])|
| 6.117771779183713|          1.0|          (34,[],[])|
| 6.117771779183713|          1.0|          (34,[],[])|
| 6.117771779183713|          1.0|          (34,[],[])|
| 6.117771779183713|          6.0|          (34,[],[])|
| 6.117771779183713|          7.0|          (34,[],[])|
| 6.117387914916791|          7.0|(34,[0,1,2,3,4,5,...|
| 6.070065355883991|          4.0|(34,[0,1,2,3,4,5,...|
| 6.115278901635902|          7.0|(34,[0,1,2,3,4,5,...|
| 6.117343458409349|          8.0|(34,[0,1,2,3,4,5,...|
| 6.117332529221051|          7.0|(34,[0,1,2,3,4,5,...|
| 6.117303445597998|          8.0|(34,[0,1,2,3,4,5,...|
| 5.593763176989088|          1.0|(34,[0,1,2,3,4,5,...|
|6.1173893267540675|          7.0|(34,[0,1,2,3,4,5,...|
| 6.117387374909611|          4.0|(34,[0,1,2,3,4

In [13]:
## Decision tree regression
print(lr_model.extractParamMap())

{Param(parent='LinearRegression_b0fafdd856ba', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LinearRegression_b0fafdd856ba', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.8, Param(parent='LinearRegression_b0fafdd856ba', name='featuresCol', doc='features column name.'): 'features', Param(parent='LinearRegression_b0fafdd856ba', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LinearRegression_b0fafdd856ba', name='labelCol', doc='label column name.'): 'attentiveness', Param(parent='LinearRegression_b0fafdd856ba', name='predictionCol', doc='prediction column name.'): 'prediction', Param(parent='LinearRegression_b0fafdd856ba', name='solver', doc='The solver algorithm for optimization. Supported options: auto, normal, l-bfgs.'): 'auto', Param(parent='LinearRegression_b0fafdd856ba', name='standard

In [6]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'attentiveness')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="attentiveness", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
dt_predictions.select('prediction', 'attentiveness', 'features').show(100)

Root Mean Squared Error (RMSE) on test data = 3.09886
+------------------+-------------+--------------------+
|        prediction|attentiveness|            features|
+------------------+-------------+--------------------+
|  3.74468085106383|          1.0|          (29,[],[])|
|  3.74468085106383|          1.0|          (29,[],[])|
|  3.74468085106383|          1.0|          (29,[],[])|
|  3.74468085106383|          1.0|          (29,[],[])|
|  3.74468085106383|          1.0|          (29,[],[])|
|  3.74468085106383|          1.0|          (29,[],[])|
|  3.74468085106383|          4.0|          (29,[],[])|
|  3.74468085106383|          6.0|          (29,[],[])|
|  3.74468085106383|          8.0|          (29,[],[])|
|  3.74468085106383|         10.0|          (29,[],[])|
|  3.74468085106383|         10.0|          (29,[],[])|
| 7.502873563218391|          7.0|(29,[0,1,2,3,4,5,...|
|3.6666666666666665|          9.0|(29,[0,1,2,3,4,5,...|
| 2.727272727272727|          6.0|(29,[0,1,2,3,4,5

In [8]:
# gradient boosted tree regression
dt_model.save('decisionTreeModel1.pckl')

In [18]:
# dt1 = DecisionTreeRegressor(featuresCol ='features', labelCol = 'attentiveness')

In [6]:
# dt_model1 = dt1.load('decisionTreeModel.pckl')
# m = DecisionTreeRegressor.load('decisionTreeModel.pckl')
from pyspark.ml.regression import DecisionTreeRegressionModel
m = DecisionTreeRegressionModel.load('decisionTreeModel.pckl')

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator

dt_predictions1 = m.transform(test_df)
dt_evaluator1 = RegressionEvaluator(
    labelCol="attentiveness", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator1.evaluate(dt_predictions1)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
dt_predictions1.select('prediction', 'attentiveness', 'features').show(100)

Root Mean Squared Error (RMSE) on test data = 2.4761
+------------------+-------------+--------------------+
|        prediction|attentiveness|            features|
+------------------+-------------+--------------------+
|3.8333333333333335|          1.0|          (34,[],[])|
|3.8333333333333335|          1.0|          (34,[],[])|
|3.8333333333333335|          1.0|          (34,[],[])|
|3.8333333333333335|          5.0|          (34,[],[])|
|3.8333333333333335|          6.0|          (34,[],[])|
|3.8333333333333335|          7.0|          (34,[],[])|
|3.8333333333333335|          8.0|          (34,[],[])|
| 7.675090252707581|          7.0|(34,[0,1,2,3,4,5,...|
| 6.497175141242938|          8.0|(34,[0,1,2,3,4,5,...|
| 6.497175141242938|          4.0|(34,[0,1,2,3,4,5,...|
| 7.675090252707581|          7.0|(34,[0,1,2,3,4,5,...|
| 7.675090252707581|          7.0|(34,[0,1,2,3,4,5,...|
|               8.2|          8.0|(34,[0,1,2,3,4,5,...|
| 7.675090252707581|          7.0|(34,[0,1,2,3,4,5,

In [13]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'attentiveness', maxIter=10)
gbt_model = gbt.fit(train_df)

gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'attentiveness', 'features').show(5)

+------------------+-------------+----------+
|        prediction|attentiveness|  features|
+------------------+-------------+----------+
|4.1745111723772865|          1.0|(34,[],[])|
|4.1745111723772865|          1.0|(34,[],[])|
|4.1745111723772865|          1.0|(34,[],[])|
|4.1745111723772865|          1.0|(34,[],[])|
|4.1745111723772865|          1.0|(34,[],[])|
+------------------+-------------+----------+
only showing top 5 rows



In [14]:
gbt_evaluator = RegressionEvaluator(
    labelCol="attentiveness", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 2.76647


In [19]:
!pip3 install opencv-python

     |████████████████████████████████| 49.1 MB 2.9 MB/s eta 0:00:01


In [4]:
import cv2

cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()

cv2.destroyAllWindows()

failed to grab frame


In [ ]:
rows = []
df2;
for i, row in df.iterrows():
    if(i<3000):
        image = row['Image ID'] + '.jpg'
        output = execCommand('cat "' + parent + image + '"| docker run -i think/affectiva')
        headers = lines[0].split(',')
        if(i==0):
            df2 = pd.DataFrame(columns=[headers])
            df2['imageId'] = 'i'
        lines = output.split('\n')
        values = lines[1].split(',')
        values.append(row['Image ID'])
#         values = np.array(lines[1].split(','))
#         values = values.reshape(1,54)
        print(len(values))
        print(len(headers))
        if(len(values) == len(headers)+1):
            df2.loc[i] = values
        else:
            print(values)